# Get/Update data

In [ ]:
import subprocess
import os

import pandas as pd
import numpy as np

from datetime import datetime
from bs4 import BeautifulSoup

import requests
import json

def get_johns_hopkins():
    git_pull = subprocess.Popen( "/usr/bin/git pull",
                         cwd = os.path.dirname( '/users/lukasboos/desktop/dokumente/python/enterprisedatascience/ads_covid-19/data/raw/COVID-19/' ),
                         shell = True,
                         stdout = subprocess.PIPE,
                         stderr = subprocess.PIPE )
    (out, error) = git_pull.communicate()


    print("Error : " + str(error))
    print("out : " + str(out))


#get world population data with html parser
def get_world_population_data():
    page = requests.get("https://www.worldometers.info/world-population/population-by-country/")
    soup = BeautifulSoup(page.content, 'html.parser')
    html_table = soup.find('table')
    all_rows = html_table.find_all('tr')
    final_data = []
    for pos,rows in enumerate(all_rows):
        col_list = [each_col.get_text(strip=True) for each_col in rows.find_all('td') ]
        final_data.append(col_list)
    final_list = pd.DataFrame(final_data).dropna()\
                    .rename(columns={1:'country', 2:'population'})
    final_list = final_list[['country','population']]
    
    #Replacing the Country List with Wrong Names
    final_list["country"] = final_list["country"].replace({'Myanmar':'Burma', 'Czech Republic (Czechia)': 'Czechia', 'DR Congo': 'Congo (Kinshasa)', 'Congo': 'Congo (Brazzaville)', 'South Korea': 'Korea, South', 'St. Vincent & Grenadines': 'Saint Vincent and the Grenadines', 'Taiwan': 'Taiwan*', 'United States': 'US','State of Palestine': 'West Bank and Gaza', 'Côte d\'Ivoire': 'Cote d\'Ivoire'})
    
    #Addition of New Data into the list : Like the Diamond Prince Kosova etc. If wouldn't be added this would lead to a key Error later.
    list_new_country = [pd.Series(['Diamond Princess', 3711], index=final_list.columns) ,
                    pd.Series(['Kosovo', 1845000], index=final_list.columns) ,
                    pd.Series(['Saint Kitts and Nevis', 52441], index=final_list.columns),
                    pd.Series(['Korea, North', 26008316], index=final_list.columns),
                    pd.Series(['Sao Tome and Principe', 211028], index=final_list.columns),
                    pd.Series(['Antarctica', 4000], index=final_list.columns),
                    pd.Series(['MS Zaandam', 1432], index=final_list.columns),
                    #took the number of participated athletes here
                    pd.Series(['Summer Olympics 2020', 11420], index=final_list.columns),
                    pd.Series(['Winter Olympics 2022', 2871], index=final_list.columns)]
    
    #Changing the indexing of country from hightest population to Alphabhetical Order
    final_list = final_list.append(list_new_country, ignore_index=True)\
                                .sort_values('country')\
                                .reset_index(drop=True)
    
    #Save list as csv
    final_list.to_csv('/users/lukasboos/desktop/dokumente/python/enterprisedatascience/ads_covid-19/data/raw/world_population_data.csv',sep=';',index=False)
    print('Number of rows: '+str(final_list.shape[0]))

    
if __name__ == '__main__':
    print('*******************Get data from the database***********************')
    get_johns_hopkins()
    get_world_population_data()

# Process data

In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime

#transform covid data into relational dataset
def store_relational_JH_data():
    data_path='/users/lukasboos/desktop/dokumente/python/enterprisedatascience/ads_covid-19/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    pd_raw=pd.read_csv(data_path)

    pd_data_base=pd_raw.rename(columns={'Country/Region':'country',
                      'Province/State':'state'})
    pd_data_base['state']=pd_data_base['state'].fillna('no')
    pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)
    pd_relational_model=pd_data_base.set_index(['state','country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )
    pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')

    pd_relational_model.to_csv('/users/lukasboos/desktop/dokumente/python/enterprisedatascience/ads_covid-19/data/processed//COVID_relational_confirmed.csv',sep=';',index=False)
    print(' Number of rows stored: '+str(pd_relational_model.shape[0]))

#process the data for all countries
def prepare_all_country_data():
    data_path='/users/lukasboos/desktop/dokumente/python/enterprisedatascience/ads_covid-19/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    df_raw_data = pd.read_csv(data_path)
    df_raw_data = df_raw_data.drop(['Lat','Long'],axis=1)
    df_raw_data = df_raw_data.rename(columns={'Country/Region':'country',
                                   'Province/State':'state'})
    df_raw_data['state'] = df_raw_data['state'].fillna('no')
    full_country_list = df_raw_data['country'].unique().tolist()
    time_idx = df_raw_data.columns[2:]
    df_analyse = pd.DataFrame({
        'date':time_idx})

    for each in full_country_list:
        df_analyse[each] = np.array(df_raw_data[df_raw_data['country']==each].iloc[:,2::].sum(axis=0))

    #Convert Date to Standard format YYYY-MM-DD
    time_idx=[datetime.strptime(each,"%m/%d/%y") for each in df_analyse.date]
    time_str=[each.strftime('%y-%m-%d') for each in time_idx]
    df_analyse['date']=time_idx
    df_analyse.to_csv('/users/lukasboos/desktop/dokumente/python/enterprisedatascience/ads_covid-19/data/processed/all_country_data.csv',sep=';',index=False)

    print(' Number of rows stored in all country data: '+str(df_analyse.shape[0]))

    
if __name__ == '__main__':
    print('*******************processing data...****************************')
    store_relational_JH_data()
    prepare_all_country_data()

# Create SIR data

In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime

from scipy import optimize
from scipy import integrate


def data_gathering():
    population_df = pd.read_csv('/users/lukasboos/desktop/dokumente/python/enterprisedatascience/ads_covid-19/data/raw/world_population_data.csv',sep=';', thousands=',')
    population_df = population_df.set_index(['country']).T
    df_analyse = pd.read_csv('/users/lukasboos/desktop/dokumente/python/enterprisedatascience/ads_covid-19/data/processed/all_country_data.csv',sep=';')

    #we need to reduce data because to much data leads to runtime error in curve fit   
    df_analyse = df_analyse[:250]
    
    country_list = df_analyse.columns[1:]

    infected_list = []

    for column in df_analyse.columns:
        infected_list.append(np.array(df_analyse[column][40:]))

    infected_df = pd.DataFrame(infected_list,index=df_analyse.columns).T
    infected_df.to_csv('/users/lukasboos/desktop/dokumente/python/enterprisedatascience/ads_covid-19/data/processed/SIR/SIR_data.csv',sep=';',index=False)
    print('Number of rows in Infected df '+str(infected_df.shape[0]))
    
    optimized_df = pd.DataFrame(columns = df_analyse.columns[1:],
                     index = ['opt_beta', 'opt_gamma', 'std_dev_error_beta', 'std_dev_error_gamma'])

    fitted_final_data = []

    global I0, N0, S0, R0
    for column in infected_df.columns[1:]:
        I0 = infected_df[column].loc[0]
        N0 = population_df[column].loc['population']
        S0 = N0-I0
        R0 = 0
        t  = np.arange(len(infected_df[column]))

        popt=[0.4,0.1]

        fit_odeint(t, *popt)


        popt, pcov = optimize.curve_fit(fit_odeint, t, infected_df[column], maxfev=5000)
        perr = np.sqrt(np.diag(pcov))



        optimized_df.at['opt_beta', column] = popt[0]
        optimized_df.at['opt_gamma', column] = popt[1]
        optimized_df.at['std_dev_error_beta', column] = perr[0]
        optimized_df.at['std_dev_error_gamma', column] = perr[1]

        fitted = fit_odeint(t, *popt)
        fitted_final_data.append(np.array(fitted))

    optimized_df.to_csv('/users/lukasboos/desktop/dokumente/python/enterprisedatascience/ads_covid-19/data/processed/SIR/optimized_SIR_data.csv',sep=';',index=False)
    fitted_SIR_data_df = pd.DataFrame(fitted_final_data,index=df_analyse.columns[1:]).T
    fitted_SIR_data_df.to_csv('/users/lukasboos/desktop/dokumente/python/enterprisedatascience/ads_covid-19/data/processed/SIR/fitted_SIR_data.csv',sep=';',index=False)
    print(' Number of rows stored in optimized df: '+str(optimized_df.shape[0]))
    print(' Number of rows stored in fitted SIR data: '+str(fitted_SIR_data_df.shape[0]))

#SIR Model
def SIR_model_t(SIRN,t,beta,gamma):
    S,I,R,N=SIRN
    dS_dt=-beta*S*I/N          
    dI_dt=beta*S*I/N-gamma*I
    dR_dt=gamma*I
    dN_dt=0
    return dS_dt,dI_dt,dR_dt,dN_dt


#fit function
def fit_odeint(t, beta, gamma):

    '''
    helper function for the integration
    '''
    return integrate.odeint(SIR_model_t, (S0, I0, R0, N0), t, args=(beta, gamma))[:,1] # we only would like to get dI


if __name__ == '__main__':
    print('*******************Calculating SIR data...*************************')
    data_gathering()

# Create Dashboard

In [ ]:
import pandas as pd
import numpy as np

import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

import plotly.graph_objects as go

import os
print(os.getcwd())


#Read data from csv
fitted_final_data_dash_df = pd.read_csv('/users/lukasboos/desktop/dokumente/python/enterprisedatascience/ads_covid-19/data/processed/SIR/fitted_SIR_data.csv',sep=';')
optimized_dash_df = pd.read_csv('/users/lukasboos/desktop/dokumente/python/enterprisedatascience/ads_covid-19/data/processed/SIR/optimized_SIR_data.csv',sep=';')
ydata_dash_df = pd.read_csv('/users/lukasboos/desktop/dokumente/python/enterprisedatascience/ads_covid-19/data/processed/SIR/SIR_data.csv',sep=';')

#create figure object
fig = go.Figure()

#create dash object
app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data
    Goal of the project is to learn data science by applying a cross industry standard process,
    it covers the full walkthrough of: automated data gathering, data transformations,
    filtering and machine learning to approximating the doubling time, and
    (static) deployment of responsive dashboard.
    '''),

    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),

    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in fitted_final_data_dash_df],
        value=['Germany', 'Italy', 'Spain'], # which are pre-selected
        multi=True
    ),

    dcc.Markdown('''
        ## SIR curve and fitted SIR curve
        '''),

    dcc.Dropdown(
    id='sir',
    options=[
        {'label': 'SIR curve and fitted SIR curve ', 'value': 'SIR_value'},
    ],
    value='SIR_value',
    multi=False
    ),

    dcc.Graph(figure=fig, id='main_window_slope')
])

@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('sir', 'value')])

def update_figure(full_country_list,show_doubling):
    traces = []
    for each in full_country_list:
        traces.append(dict(x=ydata_dash_df.date,
                                y=ydata_dash_df[each],
                                mode='markers+lines',
                                opacity=0.9,
                                line_width=2,
                                marker_size=4,
                                name=each
                        )
                )
        traces.append(dict(x=ydata_dash_df.date,
                                y=fitted_final_data_dash_df[each],
                                mode='markers+lines',
                                opacity=0.9,
                                line_width=2,
                                marker_size=4,
                                name=each+'_fitted'
                        )
                )

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis={'type': 'log',
                        'range': '[1.1,5.5]',
                        'title':'Population infected'
                        }
        )
    }


if __name__ == '__main__':
    print('***************Dashboard is ready now! Please click on the link to open the Dashboard!**********************')
    app.run_server(debug=True, use_reloader=False)